
# 3D Emulsion Droplets (PyVista)

This example loads a droplet trajectory CSV from Hugging Face and renders a clipped 3D scene with `trajkit`'s PyVista utilities.


In [ ]:

import pandas as pd
from pathlib import Path

from trajkit import TrajectorySet
from trajkit.viz.three_d import (
    TrajectoryFrameSpec,
    ensure_dataframe,
    animate_static_pyvista_scene,
)


In [ ]:

# Hugging Face CSV (public). Adjust if you mirror the data.
hf_url = "https://huggingface.co/datasets/mehdi/trajkitData/resolve/main/data/csv/EmulsionSGM_Traj.csv"
# Where to write the movie
output_mp4 = Path("sgm_scene.mp4")


In [ ]:

# Load and clean the droplet table
# Expected columns: id, t, x, y, z, r

df = pd.read_csv(hf_url)
if df.columns[0].startswith("Unnamed"):
    df = df.drop(columns=df.columns[0])
# Use a frame column for visualization (rename t -> frame)
if "frame" not in df.columns and "t" in df.columns:
    df = df.rename(columns={"t": "frame"})

spec = TrajectoryFrameSpec(id_col="id", t_col="frame", x_col="x", y_col="y", z_col="z", r_col="r")
df = ensure_dataframe(df, spec)
df.head()


In [ ]:

# Optional: build a TrajectorySet for downstream analysis
trajset = TrajectorySet.from_dataframe(
    df,
    dataset_id="emulsion_sgm",
    track_id_col=spec.id_col,
    position_cols=[spec.x_col, spec.y_col, spec.z_col],
    frame_col=spec.t_col,
    frame_rate_hz=1.0,
    frame_feature_cols=[spec.r_col],
)
trajset.summary_table().head()


In [ ]:

# Render a boxed, clipped scene. Adjust bounds to your field of view.
bounds = (0, 505, 0, 410, 0, 505)
animate_static_pyvista_scene(
    df,
    spec=spec,
    bounds=bounds,
    output=str(output_mp4),
    fps=20,
    clip_to_bounds=True,
    clip_strategy="box",  # try "boolean" if you prefer boolean clipping
    camera_position="iso",
    camera_distance_scale=1.4,
    background="white",
    fill_clipped_holes=True,
    fill_holes_radius_scale=0.25,
)
output_mp4
